# Advent of Code 2022 Solutions

In [1]:
import pandas as pd
from aoc import get_input_data

## Day 1

In [2]:
data = get_input_data(1)

s = pd.Series([sum(map(int, chunk.split('\n'))) for chunk in data.split('\n\n')])

# Part 1
print(s.max())

# Part 2
print(s.nlargest(3).sum())

70698
206643


## Day 2

In [3]:
data = """A Y
B X
C Z"""

data

'A Y\nB X\nC Z'

In [4]:
data = get_input_data(2)

### Part 1

First Column
> A for Rock, B for Paper, and C for Scissors.

Second column
> X for Rock, Y for Paper, and Z for Scissors

Scoring
> (1 for Rock, 2 for Paper, and 3 for Scissors)<br>
> (0 if you lost, 3 if the round was a draw, and 6 if you won)

In [5]:
d = {
    'A': {
        'X': (1, 3),
        'Y': (2, 6),
        'Z': (3, 0)
    },
    'B': {
        'X': (1, 0),
        'Y': (2, 3),
        'Z': (3, 6)
    },
    'C': {
        'X': (1, 6),
        'Y': (2, 0),
        'Z': (3, 3)
    },
}

score = 0

for x in data.split('\n'):
    p1, p2 = x.split()
    points, outcome = d[p1][p2]
    score += points + outcome

score

12586

### Part 2

> X means you need to lose, Y means you need to end the round in a draw, and Z means you need to win

In [6]:
d_points = {
    'A': 1, 'B': 2, 'C': 3,
    'X': 1, 'Y': 2, 'Z': 3
}
d_win_lose = {'A': ('Y', 'Z'), 'B': ('Z', 'X'), 'C': ('X', 'Y')}


score = 0

for x in data.split('\n'):
    p1, p2 = x.split()
    
    win, lose = d_win_lose[p1]

    # Lose
    if p2 == 'X':
        outcome = 0
        points = d_points[lose]
    
    # Draw
    elif p2 == 'Y':
        outcome = 3
        points = d_points[p1]
        
    # Win
    else:
        outcome = 6
        points = d_points[win]
    score += outcome + points

score

13193

---
## Day 3
### Part 1

In [7]:
data = """vJrwpWtwJgWrhcsFMMfFFhFp
jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL
PmmdzqPrVvPwwTWBwg
wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn
ttgJtRGJQctTZtZT
CrZsJsPPZsGzwwsLwLmpwMDw"""

data = data.split('\n')
data

['vJrwpWtwJgWrhcsFMMfFFhFp',
 'jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL',
 'PmmdzqPrVvPwwTWBwg',
 'wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn',
 'ttgJtRGJQctTZtZT',
 'CrZsJsPPZsGzwwsLwLmpwMDw']

In [8]:
data = get_input_data(3)
data = data.split('\n')

In [9]:
from string import ascii_lowercase, ascii_uppercase

# Build letter priority dict
priorities = {l: i for i, l in enumerate(ascii_lowercase, 1)}
priorities.update({l: i for i, l in enumerate(ascii_uppercase, 27)})
print(priorities)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 35, 'J': 36, 'K': 37, 'L': 38, 'M': 39, 'N': 40, 'O': 41, 'P': 42, 'Q': 43, 'R': 44, 'S': 45, 'T': 46, 'U': 47, 'V': 48, 'W': 49, 'X': 50, 'Y': 51, 'Z': 52}


In [10]:
def split_rucksack(rucksack):
    mid = len(rucksack) // 2
    return rucksack[:mid], rucksack[mid:]

def get_common_items(compartment1, compartment2):
    c1 = set(compartment1)
    c2 = set(compartment2)
    return c1 & c2

In [11]:
score = 0

for rucksack in data:
    compartments = split_rucksack(rucksack)
    common_items = get_common_items(*compartments)
    for item in common_items:
        score += priorities.get(item)
score

8515

### Part 2

In [12]:
def get_common_items_v2(*rucksacks):
    items = [set(rucksack) for rucksack in rucksacks]
    return set.intersection(*items)


In [13]:
score = 0
n_group = 3

for i in range(0, len(data), n_group):
    group_rucksacks = data[i: i + n_group]
    common_items = get_common_items_v2(*group_rucksacks)
    for item in common_items:
        score += priorities.get(item)
score

2434

## Day 4
### Part 1

In [14]:
data = '''2-4,6-8
2-3,4-5
5-7,7-9
2-8,3-7
6-6,4-6
2-6,4-8'''


In [15]:
def process_line(line):
    ranges = dict(enumerate(line.split(',')))
    for i, rng in ranges.items():
        start, end = map(int, rng.split('-'))
        rng = range(start, end+1)
        ranges[i] = rng
    return tuple(ranges.values())

def order_ranges(*ranges):
    return sorted(ranges, key=lambda x: x.start - x.stop)

def is_contained(outer, inner):
    if (outer.start <= inner.start) and (outer.stop >= inner.stop):
        return True
    return False


In [16]:
data = get_input_data(4)

In [17]:
%%time
count = 0

for line in data.split('\n'):
    rng1, rng2 = process_line(line)
    ordered = order_ranges(rng1, rng2)
    if is_contained(*ordered):
        count += 1
count

CPU times: user 5.95 ms, sys: 138 µs, total: 6.09 ms
Wall time: 6.09 ms


431

### Part 2

In [18]:
def overlaps(rng1, rng2):
    if set(rng1).intersection(rng2):
        return True
    if set(rng2).intersection(rng1):
        return True
    return False

In [19]:
%%time

count = 0

for line in data.split('\n'):
    rng1, rng2 = process_line(line)
    if overlaps(rng1, rng2):
        count += 1
count

CPU times: user 5.72 ms, sys: 8 µs, total: 5.72 ms
Wall time: 5.73 ms


823

---
## Day 5
### Part 1

In [20]:
data = '''    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2'''

In [21]:
import re

def process_stack_line(line):
    step = 4
    crates = {}
    
    for i, idx in enumerate(range(0, len(line), step), 1):
        crate = line[idx:idx+step]
        if crate.startswith('['):
            crates[i] = crate[1]
    return crates

def process_data(data):
    stacks = {}
    lines = iter(data.split('\n'))
    line = ''
    
    while not line.startswith(' 1'):
        line = next(lines)
        crates = process_stack_line(line)
        for n, crate in crates.items():
            stacks.setdefault(n, []).append(crate)
    return stacks, list(lines)[1:]

def parse_instruction(instruction):
    pat = re.compile(r'move (\d+) from (\d+) to (\d+)')
    groups = list(map(int, pat.match(instruction).groups()))
    return groups

def move_crates(stacks, n, from_, to, model=9000):
        
    crates = []
    for i in range(n):
        if stacks[from_]:
            crate = stacks[from_].pop(0)
            crates.append(crate)

    if model == 9000:
        for crate in crates:
            stacks[to].insert(0, crate)
    elif model == 9001:
        stacks[to] = crates + stacks[to]
    else:
        raise ValueError('model can be 9000 or 9001')


def get_answer(stacks):
    result = ''
    for i in range(1, len(stacks) + 1):
        result += stacks[i][0]
    return result

def render_stacks(stacks):
    for i in range(1, len(stacks) + 1):
        print(f'{i:>2} - {stacks[i]}')

In [22]:
data = get_input_data(5)

In [23]:
stacks, instructions = process_data(data)

for instruction in instructions:
    
    n, from_, to = parse_instruction(instruction)
    move_crates(stacks, n, from_, to)

get_answer(stacks)

'VQZNJMWTR'

### Part 2

In [24]:
stacks, instructions = process_data(data)

for instruction in instructions:
    
    n, from_, to = parse_instruction(instruction)
    move_crates(stacks, n, from_, to, model=9001)

get_answer(stacks)

'NLCDCLVMQ'

---
## Day 6
### Part 1

In [25]:
data = 'mjqjpqmgbljsphdztnvjfqwrcgsmlb'

In [26]:
data = get_input_data(6)

In [27]:
def get_start_of_packet(data, n=4):

    for i in range(len(data)):
        chars = set(data[i:i+n])
        if len(chars) == n:
            return i + n

get_start_of_packet(data)

1480

### Part 2

In [28]:
get_start_of_packet(data, n=14)

2746

---
## Day 7
### Part 1

In [29]:
data = '''$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k'''

In [30]:
class Directory():
    def __init__(self, name, parent):
        self.name = name
        self.parent = parent
        self._children = {}
        self.files = {}
        
    @property
    def children(self):
        return list(self._children.values())
        
    @property
    def root(self):
        root_ = self
        while root_.parent is not None:
            root_ = root_.parent
        return root_
    
    @property
    def depth(self):
        depth_ = 0
        root_ = self
        while root_.parent is not None:
            root_ = root_.parent
            depth_ += 1
        return depth_

    def add_file(self, name, size):
        self.files[name] = int(size)
        
    def add_child(self, name):
        self._children[name] = Directory(name, self)
        
    def get_size(self):
        return sum(self.files.values())
    
    def get_total_size(self):
        total = self.get_size()
        
        for child in self._children.values():
            total += child.get_total_size()
        return total
    
    def cd(self, name):
        if name == '..':
            return self.parent
        if name == '/':
            return self.root
        return self._children[name]

    def process_command(self, command):
        if command == '$ ls':
            return self
        elif command.startswith('$ cd'):
            name = command.split()[-1]
            return self.cd(name)
        elif command.startswith('dir'):
            _, name = command.split()
            self.add_child(name)
        elif command[0].isdigit():
            size, name = command.split()
            self.add_file(name, size)
        return self

    def __repr__(self):
        return (f"Directory('{self.name}', root='{self.root.name}', "
                f"total_size={self.get_total_size()}, depth={self.depth})")

    def __str__(self):
        padding = '  ' * self.depth
        output = [f"{padding}-{self.name} (dir) - total_size: {self.get_total_size()}"]
        for name, size in self.files.items():
            output.append(f"{padding}  {name} ({size})")
        for child in self.children:
            output.append(f"{child.__str__()}")
        return '\n'.join(output)



In [31]:
data = get_input_data(7)

In [32]:
# Build the directory tree
tree = Directory('/', parent=None)
for command in data.split('\n'):
    tree = tree.process_command(command)

# reset back to root
tree = tree.root

# Traverse tree and get sizes
to_check = {tree}
seen = set()
total_sizes = []
max_size = 100000

while to_check:
    dir_ = to_check.pop()
    seen.add(dir_)
    children = set(dir_.children)
    to_check.update(children.difference(seen))
    total_sizes.append(dir_.get_total_size())

# Filter sizes and sum for result
filtered_sizes = [x for x in total_sizes if x <= max_size]
sum(filtered_sizes)


1297159

### Part 2

In [33]:
disk_space = 70000000
space_needed = 30000000

total_used = max(total_sizes)
unused_space = disk_space - total_used
target = space_needed - unused_space

min([x for x in total_sizes if x > target])

3866390